<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`


## Presentado por: Andrés Felipe Murillo

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [ ]:
import pandas as pd
import tweepy

In [ ]:
#leer las llaves
keys = pd.read_csv("C:/Users/AndresFelipe/Documents/Talleres NLP/Taller 10 twitter/mis_llaves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [ ]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
### Escogí los tuits referentes a Steph Curry
### ingrese las coordenadas de San Francisco - California - USA
tweets = tweepy.Cursor(api.search,
                       q=['Steph Curry -RT'],
                       lang='es',
                       geocode='37.7272,-123.032,5000km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(2000)

In [ ]:
# Objeto generador
t = [tweet for tweet in tweets]

In [ ]:
t[0]

Status(_api=<tweepy.api.API object at 0x000000D14B019520>, _json={'created_at': 'Thu May 20 15:53:43 +0000 2021', 'id': 1395407398957944838, 'id_str': '1395407398957944838', 'full_text': 'Todos se quedaron con la última jugada de su majestad Lebron anoche, pero que partidazo de Steph Curry, uno más, nada nuevo!', 'truncated': False, 'display_text_range': [0, 124], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'es', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 362867631, 'id_str': '362867631', 'name': '*Carlos Barros*', 'screen_name': 'ElCarly95', 'location': 'Barranquilla, Colombia', 'description': 'JUNIOR DEL ALMA.', 'url': None, 'entities': {'description': {'urls': []}

In [ ]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)

In [ ]:
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweets,date,city,user_location,id,link
0,Todos se quedaron con la última jugada de su m...,2021-05-20 15:53:43,Bogotá,"Barranquilla, Colombia",1395407398957944838,https://twitter.com/i/web/stuatus/139540739895...
1,Me van a bardear pero Steph Curry se preocupó ...,2021-05-20 13:54:53,Bogotá,Argentina,1395377490466643968,https://twitter.com/i/web/stuatus/139537749046...
2,Hay que identificar la diferencia entre un jug...,2021-05-20 13:19:30,Bogotá,Ecuador,1395368585325842436,https://twitter.com/i/web/stuatus/139536858532...
3,Mención aparte a Steph Curry. Un MVP por todo ...,2021-05-20 12:58:36,Bogotá,"Guayaquil, Guayas, Ecuador.",1395363326343921669,https://twitter.com/i/web/stuatus/139536332634...
4,están ni cerca de contar su impacto en el part...,2021-05-20 06:39:55,Bogotá,"Los Angeles, CA",1395268028565123077,https://twitter.com/i/web/stuatus/139526802856...


In [ ]:
data.shape

(119, 6)

In [ ]:
import re

In [ ]:
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

In [ ]:
def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions

In [ ]:
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions
0,Todos se quedaron con la última jugada de su m...,2021-05-20 15:53:43,Bogotá,"Barranquilla, Colombia",1395407398957944838,https://twitter.com/i/web/stuatus/139540739895...,Todos se quedaron con la última jugada de su m...,[],[],[]
1,Me van a bardear pero Steph Curry se preocupó ...,2021-05-20 13:54:53,Bogotá,Argentina,1395377490466643968,https://twitter.com/i/web/stuatus/139537749046...,Me van a bardear pero Steph Curry se preocupó ...,[https://t.co/xo5XHWFnqS],[],[]
2,Hay que identificar la diferencia entre un jug...,2021-05-20 13:19:30,Bogotá,Ecuador,1395368585325842436,https://twitter.com/i/web/stuatus/139536858532...,Hay que identificar la diferencia entre un jug...,[],[],[]
3,Mención aparte a Steph Curry. Un MVP por todo ...,2021-05-20 12:58:36,Bogotá,"Guayaquil, Guayas, Ecuador.",1395363326343921669,https://twitter.com/i/web/stuatus/139536332634...,Mención aparte a Steph Curry. Un MVP por todo ...,[],[],[]
4,están ni cerca de contar su impacto en el part...,2021-05-20 06:39:55,Bogotá,"Los Angeles, CA",1395268028565123077,https://twitter.com/i/web/stuatus/139526802856...,están ni cerca de contar su impacto en el part...,[],[],[]
...,...,...,...,...,...,...,...,...,...,...
114,Último minuto le doy el balón a Steph Curry… S...,2021-05-15 01:57:44,Bogotá,Piura,1393385074658271233,https://twitter.com/i/web/stuatus/139338507465...,Último minuto le doy el balón a Steph Curry… S...,[],[#NBAxESPN],[]
115,Steve Kerr dice que Steph Curry ha estado juga...,2021-05-14 23:53:09,Bogotá,"Oakland, CA",1393353724840579073,https://twitter.com/i/web/stuatus/139335372484...,Steve Kerr dice que Steph Curry ha estado juga...,[https://t.co/40cy66X76J],[],[]
116,@GSWarriorsLATAM Fue lo más ridículo que leí d...,2021-05-14 16:38:51,Bogotá,Panamá,1393244427318087686,https://twitter.com/i/web/stuatus/139324442731...,Fue lo más ridículo que leí de ESPN y si cons...,[],[],[@GSWarriorsLATAM]
117,Tenemos una discusión corriendo en el grupo de...,2021-05-14 14:23:42,Bogotá,"Miami, FL",1393210414553522182,https://twitter.com/i/web/stuatus/139321041455...,Tenemos una discusión corriendo en el grupo de...,[],[],[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [ ]:
#Series de tiempo

In [ ]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp

,hora,cnt
80,00:59:00,1
88,01:19:00,1
41,01:30:00,1
76,01:57:00,1
62,02:02:00,1
...,...,...
86,23:10:00,1
71,23:11:00,1
45,23:29:00,1
4,23:40:00,3


In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [ ]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)